## **1. List of Available Pretrained Models**


#### **Classification Models**

Use:

```python
from torchvision import models

# List all classification models
print(dir(models))
```

**Common pretrained classification models**:

| Model Family | Model Names                                                  |
| ------------ | ------------------------------------------------------------ |
| VGG          | `vgg11`, `vgg13`, `vgg16`, `vgg19`, and their `_bn` variants |
| ResNet       | `resnet18`, `resnet34`, `resnet50`, `resnet101`, `resnet152` |
| DenseNet     | `densenet121`, `densenet161`, `densenet169`, `densenet201`   |
| MobileNet    | `mobilenet_v2`, `mobilenet_v3_large`, `mobilenet_v3_small`   |
| EfficientNet | `efficientnet_b0` to `efficientnet_b7`                       |
| ViT          | `vit_b_16`, `vit_b_32`, `vit_l_16`, etc.                     |
| ConvNeXt     | `convnext_tiny`, `convnext_base`, etc.                       |
| RegNet       | `regnet_y_400mf`, `regnet_y_1_6gf`, etc.                     |
| SqueezeNet   | `squeezenet1_0`, `squeezenet1_1`                             |

---

#### **Segmentation Models**

Available under `torchvision.models.segmentation`:

```python
from torchvision.models import segmentation
print(dir(segmentation))
```

**Popular segmentation models**:

* `fcn_resnet50`
* `fcn_resnet101`
* `deeplabv3_resnet50`
* `deeplabv3_resnet101`
* `lraspp_mobilenet_v3_large`
---

##  2. **Input and Output Size of Pretrained Network**

Most models expect input images of size **`3 x H x W`**, where:

* **3** is the number of color channels (RGB).
* **H, W** must be at least **224 x 224** for classification models.
* Some segmentation models require specific input sizes to avoid dimensionality issues due to downsampling.

Here’s how you can check the **minimum input size** programmatically:

```python
import torch
from torchvision import models
from torchvision.models import vgg19

model = models.vgg19_bn(weights=models.VGG19_BN_Weights.IMAGENET1K_V1)

# Dummy input to test
x = torch.randn(1, 3, 224, 224)  # Batch size 1
out = model(x)
print(out.shape)  # Output: [1, 1000] -> 1000 classes
```

**General Input Size Guide**:

| Model        | Recommended Input Size                         |
| ------------ | ---------------------------------------------- |
| VGG          | 224 × 224                                      |
| ResNet       | 224 × 224                                      |
| DenseNet     | 224 × 224                                      |
| EfficientNet | Varies per version (e.g., B0: 224, B7: 600)    |
| ViT          | Typically 224 × 224 (patch size varies)        |
| DeepLabV3    | 513 × 513 (can vary, should be divisible by 8) |
| FCN          | 224 × 224 or larger                            |

---




## **3. Determining the required input size**
---

#### 3.1. **Use TorchVision Documentation or Model Summary**

The [official PyTorch documentation](https://pytorch.org/vision/stable/models.html) lists **default input sizes** for each pretrained model.

---

####  3.2 **Inspect the Model Internals**

For most models, like `resnet18`, you can inspect how many times the input is halved due to pooling/stride:

```python
from torchvision import models
from torchinfo import summary  # pip install torchinfo

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
summary(model, input_size=(1, 3, 224, 224))
```

---

####  3.3 **Render the Model Diagram (Visualization)**

```python
input=torch.randn(size=[1,3,128,128])

resnet18_graph=torchviz.make_dot(resnet18(input) ,dict(resnet18.named_parameters()))
resnet18_graph.format='svg'
resnet18_graph.save('images/resnet18_graph')
resnet18_graph.render()
```

## **4. Input Image Size Different From The Pretrained Model Input**

If your input image size is **different** from what the pretrained model expects, you have **two main options**, depending on your task:


#### **4.1 Resize your input image to match the model**


```python
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Match model's expected input
    transforms.ToTensor(),
])
```

---
#### **4.2 Adapt the model to your image size**

**Advanced — use only if resizing hurts performance or semantics.**

- Use **adaptive pooling** in place of fixed `AvgPool2d` or `Linear` assumptions (e.g., in custom CNNs):

```python
nn.AdaptiveAvgPool2d((1, 1))  # Allows any input size
```

- **Replace classifier layers** if needed:

If your model fails because of mismatched `in_features` in `Linear`, do:

```python
# Forward pass dummy input to find flattened size
dummy_input = torch.randn(1, 3, your_H, your_W)
features = model.features(dummy_input)  # or model.backbone for ResNet
flattened_size = features.view(1, -1).shape[1]

# Replace classifier accordingly
model.classifier = nn.Sequential(
    nn.Linear(flattened_size, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes),
)
```

When to use:

* You're training **from scratch** or fine-tuning a model deeply.
* Your data has **very different resolution** (e.g. medical images 512x512).
* You want to **preserve spatial details** for segmentation/localization.

---


## **5. Customizing of a Pretrained Network Classifier**

First you have to know the parameters of your network, for instance:

```python
model_vgg19_bn = models.vgg19_bn(weights=models.VGG19_BN_Weights.IMAGENET1K_V1)
print(model_vgg19_bn)
```

This will give you the entire model features (covnet layer) + fully connected layer:

```bash
(features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    .
    .
    .
    
    (50): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (51): ReLU(inplace=True)
    (52): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

(classifier): Sequential(
    (0): Linear(in_features=25088, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=4096, out_features=1000, bias=True)

```

or 


```python
for param in model_vgg19_bn.features.parameters():
    print(param.shape)
```

gives you features (covnet layer):

```bash
torch.Size([64, 3, 3, 3])
torch.Size([64])
.
.
.
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512])
torch.Size([512])
```


For **ResNet18**, we have only covnet layer and 1 fully connected layer, input is 512 and output 1000 classes:

```python
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
print("resnet18 input size: ", resnet18.fc.in_features)
print("resnet18 output size: ", resnet18.fc.out_features)
```

```bash
resnet18 input size:  512
resnet18 output size:  1000
```
---

#### ResNet18

Let’s say your dataset has **3 classes**, and you want to reuse a pretrained model for **image classification**.

---

**Freezing Feature Extractor**

```python
# Freeze all layers so we don't train them
for param in resnet18.parameters():
    param.requires_grad = False
```

---

**Replace the Final Classifier**


```python
num_classes = 3  # your problem
in_features = model.fc.in_features

resnet18.fc = nn.Linear(in_features, num_classes)  # new classifier layer
```


**Optimizer for parameters**

```python
optimizer = optim.Adam(resnet18.fc.parameters(), lr=0.001)
```



#### VGG19

If you're using **VGG19**, it would look like this instead:

```python
model_vgg19_bn = models.vgg19_bn(weights=models.VGG19_BN_Weights.IMAGENET1K_V1)
for param in model_vgg19_bn.features.parameters():
    param.requires_grad = False

model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
```

**Optimizer for parameters**


```python
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
```

OR if you only want to train the last layer (as in transfer learning):

```python
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)
```

- `model.classifier` is a Sequential block of layers.
- `model.classifier[6]` is the last Linear layer you replaced.  (preferred)


---

**Train the Model**

```python
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):  # train for 5 epochs
    model.train()
    for images, labels in train_loader:  # assume you have train_loader
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
```

---

**Optional: Fine-Tune Some Layers**

Instead of freezing all feature layers, you can unfreeze a few:

```python
# Unfreeze last few layers of ResNet
for name, param in model.named_parameters():
    if "layer4" in name:  # last block of ResNet
        param.requires_grad = True
```

---

**Evaluation**

```python
model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
```

---




####  Tips for Transfer Learning

1. **Classification**

   * Replace the last linear layer (`model.fc` for ResNet, `model.classifier` for VGG) with:

     ```python
     model.fc = nn.Linear(in_features, num_classes)
     ```
   * Freeze earlier layers:

     ```python
     for param in model.parameters():
         param.requires_grad = False
     ```

2. **Segmentation**

   * Output has shape `[B, C, H, W]`, where `C = num_classes`
   * You can replace the classifier head:

     ```python
     model.classifier[4] = nn.Conv2d(in_channels, num_classes, kernel_size=1)
     ```

Would you like an example of transfer learning for classification and segmentation with a specific model?

## Fine-Tuning

1. **Feature extraction** = freeze everything, train new classifier head.
2. **Fine-tuning** = unfreeze some or all layers, retrain using your tumor dataset to adapt its internal filters to your domain.




## **When to  Learn Feature maps**

---

**Transfer Learning Modes**

| Mode                               | Freeze Feature Layers? | Fine-Tune Feature Layers?                     | Train Classifier? | When to Use                                                                                      |
| ---------------------------------- | ---------------------- | --------------------------------------------- | ----------------- | ------------------------------------------------------------------------------------------------ |
| **1. Feature Extraction (Frozen)** | ✅ Yes                  | ❌ No                                          | ✅ Yes             | When dataset is **small** and **similar** to ImageNet                                            |
| **2. Fine-Tuning Last Block**      | 🚫 No (partial)        | ✅ Last layers only (e.g., `layer4` in ResNet) | ✅ Yes             | When dataset is **moderate in size** and **domain-shifted**                                      |
| **3. Full Fine-Tuning**            | ❌ No                   | ✅ All conv layers                             | ✅ Yes             | When dataset is **large** or **significantly different** from ImageNet (e.g. medical, satellite) |
| **4. Training from Scratch**       | ❌ N/A                  | ✅ All layers randomly initialized             | ✅ Yes             | When you have a **huge custom dataset** and **no pretraining** is applicable                     |

---

## 🔧 How to Control It in Code

### ✅ 1. **Feature Extraction Only**:

```python
for param in model.parameters():
    param.requires_grad = False

# Replace and train classifier only
model.fc = nn.Linear(model.fc.in_features, num_classes)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
```

---

### ✅ 2. **Fine-Tune Last Feature Block Only (ResNet)**

```python
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# optimizer with fine-tuned + classifier params
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(trainable_params, lr=1e-4)
```

---

### ✅ 3. **Full Fine-Tuning**

```python
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-4)
```

---

## 🎯 Best Practices

* ✅ **Small dataset?** → Feature extraction only
* ✅ **Same domain?** → Freeze early layers, fine-tune late ones
* ✅ **Different domain or high resolution?** → Fine-tune more (or all)
* ✅ **Training slow?** → Freeze more layers, reduce learning rate

---

Would you like a utility function like `set_trainable_layers(model, mode='feature', last_block='layer4')` to automate this logic for ResNet or VGG?
